In [1]:
import numpy as np
import rioxarray
from xrspatial.zonal import stats
from rasterio.windows import from_bounds
import rasterio
import xarray as xr
from dask.distributed import Client, LocalCluster

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\cupy\_environment.py:216: UserWarning: CUDA path could not be detected. Set CUDA_PATH environment variable if CuPy fails to load.
  warnings.warn(


In [2]:
cluster = LocalCluster(n_workers=8)
client = Client(cluster)
client.dashboard_link

'http://127.0.0.1:8787/status'

In [3]:
LandscapeLayer = 'O:/PRIV/CPHEA/PESD/COR/CORFILES/Geospatial_Library_Projects/StreamCat/LandscapeRasters/QAComplete/CanalsDitches.tif'
inZoneData = 'O:/PRIV/CPHEA/PESD/COR/CORFILES/Geospatial_Library_Resource/Physical/HYDROLOGY/NHDPlusV21/NHDPlusNE/NHDPlus01/NHDPlusCatchment/cat'

In [4]:
izd_array = rioxarray.open_rasterio(inZoneData, chunks=True).sel(band=1).drop_vars('band')
izd_array

<xarray.DataArray (y: 31405, x: 25845)> Size: 3GB
dask.array<getitem, shape=(31405, 25845), dtype=int32, chunksize=(1296, 25845), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 207kB 1.644e+06 1.644e+06 ... 2.419e+06 2.419e+06
  * y            (y) float64 251kB 3.143e+06 3.143e+06 ... 2.2e+06 2.2e+06
    spatial_ref  int32 4B 0
Attributes:
    _FillValue:    -2147483647
    scale_factor:  1.0
    add_offset:    0.0

In [5]:
transform = izd_array.rio.transform()
bounds = izd_array.rio.bounds()
window = from_bounds(*bounds, transform)
window

Window(col_off=0.0, row_off=0.0, width=25845.000000000007, height=31405.0)

In [6]:
with rasterio.open(LandscapeLayer) as src:
    ll_array = src.read(1, window=window)
ll_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [9]:
ll_xarray =  xr.DataArray(
        data=ll_array,
        dims=['y', 'x']
    ).chunk(izd_array.chunksizes)
ll_xarray

<xarray.DataArray (y: 31405, x: 25845)> Size: 812MB
dask.array<xarray-<this-array>, shape=(31405, 25845), dtype=uint8, chunksize=(1296, 25845), chunktype=numpy.ndarray>
Dimensions without coordinates: y, x

In [10]:
outTable = stats(izd_array, ll_xarray, nodata_values=izd_array.rio.nodata)
outTable

,zone,mean,max,min,sum,std,var,count
npartitions=1,,,,,,,,
,int32,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...


In [11]:
final_df = outTable.compute()

C:\Users\thudso02\AppData\Roaming\Python\Python312\site-packages\distributed\client.py:3362: UserWarning: Sending large graph of size 774.12 MiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(


In [12]:
final_df.head()

,zone,mean,max,min,sum,std,var,count
0,-2147483647,0.00034,1.0,0.0,211763.0,0.018428,0.00034,623390355.0
1,1809703,0.00000,0.0,0.0,0.0,0.000000,0.00000,2567.0
2,1809704,0.00000,0.0,0.0,0.0,0.000000,0.00000,4381.0
3,1809705,0.00000,0.0,0.0,0.0,0.000000,0.00000,6555.0
4,1809706,0.00000,0.0,0.0,0.0,0.000000,0.00000,3125.0


In [ ]:
final_df.to_csv('medium_res_stats_dask_test.csv')